# Full Dataset KMeans

In [1]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import pandas as pd
from dask import delayed, persist
import numpy as np
from dask.distributed import Client
from glob import glob
import re
import nltk
import dask
import csv
csv.field_size_limit(10000000)


pd.set_option('display.max_columns', None) 

%matplotlib inline

csv.field_size_limit()

10000000

In [2]:
client = Client(n_workers=3, threads_per_worker=2, memory_limit='1GB')
client

Client Scheduler: tcp://127.0.0.1:52491 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 6 Memory: 3.00 GB


In [15]:
client.close()

In [3]:
import dask.array as da

x = da.random.random((10000, 10000), chunks='16 MiB')
x

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1250, 1250), chunktype=numpy.ndarray>

In [4]:
y = (x + x.T) - x.mean(axis=0)
y.sum().compute()

49998339.368165314

In [5]:
del y
del x

In [9]:
best_list = ['JobID', 'CleanJobTitle', 'CanonCity', 'CanonState', 'JobDate', 'JobText', 'Source', 'CanonEmployer',
             'Latitude', 'Longitude', 'CanonIntermediary', 'CanonJobTitle', 'CanonCounty', 'DivisionCode', 'MSA', 'LMA',
             'InternshipFlag', 'ConsolidatedONET', 'CanonSkillClusters', 'CanonSkills', 'IsDuplicate', 'CanonMinimumDegree', 
             'CanonRequiredDegrees', 'CIPCode', 'MinExperience', 'ConsolidatedInferredNAICS', 'BGTOcc', 'MaxAnnualSalary',
             'MaxHourlySalary', 'MinAnnualSalary', 'MinHourlySalary', 'YearsOfExperience', 'CanonJobHours', 'CanonJobType',
             'CanonPostalCode', 'CanonYearsOfExperienceCanonLevel', 'CanonYearsOfExperienceLevel', 'ConsolidatedTitle', 
             'Language', 'BGTSubOcc', 'ConsolidatedDegreeLevels', 'MaxDegreeLevel', 'MinDegreeLevel']

In [6]:
dtypes={'JobID': np.str, 'CleanJobTitle': np.str, 'JobDomain': np.str, 
        'CanonCity': np.str, 'CanonCountry': np.str, 'CanonState': np.str, 
        'JobText': np.str, 'JobURL': np.str, 'PostingHTML': np.float64, 
        'Source': np.str, 'JobReferenceID': np.str, 'Email': np.str, 
        'CanonEmployer': np.str, 'Latitude': np.str, 'Longitude': np.str, 
        'CanonIntermediary': np.str, 'Telephone': np.str, 'CanonJobTitle': np.str, 
        'CanonCounty': np.str, 'DivisionCode': np.str, 'MSA': np.str, 'LMA': np.str,
        'InternshipFlag': np.str, 'ConsolidatedONET': np.str, 'CanonCertification': np.str, 
        'CanonSkillClusters': np.str, 'CanonSkills': np.str, 'IsDuplicate': np.str, 
        'IsDuplicateOf': np.float64, 'CanonMaximumDegree': np.str, 'CanonMinimumDegree': np.str, 
        'CanonOtherDegrees': np.str, 'CanonPreferredDegrees': np.str,
        'CanonRequiredDegrees': np.str, 'CIPCode': np.str, 'StandardMajor': np.str, 
        'MaxExperience': np.str, 'MinExperience': np.str, 'ConsolidatedInferredNAICS': np.str, 
        'BGTOcc': np.str, 'MaxAnnualSalary': np.str, 'MaxHourlySalary': np.str, 
        'MinAnnualSalary': np.str, 'MinHourlySalary': np.str, 'YearsOfExperience': np.str, 
        'CanonJobHours': np.str, 'CanonJobType': np.str, 'CanonPostalCode': np.str, 
        'CanonYearsOfExperienceCanonLevel': np.str, 'CanonYearsOfExperienceLevel': np.str, 
        'ConsolidatedTitle': np.str, 'Language': np.str, 'BGTSubOcc': np.str, 'JobDate': np.str,
        'ConsolidatedDegreeLevels': np.str, 'MaxDegreeLevel': np.str, 'MinDegreeLevel': np.str,
        
        'charac_text_len': np.int16,
        'len_job_text': np.int16,
        'titles_bi_1000': np.int16,
        'titles_bi_500': np.int16,
        'titles_uni_1000': np.int16,
        'titles_uni_500': np.int16
                       }

In [7]:
ddf = dd.read_csv('~/Dropbox/Burning Glass/Data/random_data/km_500and1000k_clean_dataset2.csv', 
                 engine='python', 
                 dtype=dtypes,
                 sample=2_500_000,
                #  encoding='latin-1',
                #  encoding='utf-8',
                #  assume_missing=True,
                 error_bad_lines=False,
                #  blocksize=None,
                #  usecols=best_list,
#                  parse_dates=['JobDate']
                )
ddf

,Language,CanonCity,CanonState,CleanJobTitle,JobDate,JobID,Latitude,Longitude,CanonPostalCode,CanonCounty,DivisionCode,LMA,MSA,CanonEmployer,CanonJobTitle,ConsolidatedONET,CanonIntermediary,InternshipFlag,Source,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,CanonJobHours,CanonJobType,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedDegreeLevels,ConsolidatedTitle,MaxDegreeLevel,MinDegreeLevel,BGTSubOcc,YearsOfExperience,titles_uni_500,titles_bi_500,titles_uni_1000,titles_bi_1000,dist__bi_1000,dist__uni_500,dist__bi_500,dist__uni_1000,dist__bi_1000_dummy,dist__uni_500_dummy,dist__bi_500_dummy,dist__uni_1000_dummy,clean_text,charac_text_len,len_job_text
npartitions=58,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,int16,int16,int16,int16,float64,float64,float64,float64,int64,int64,int64,int64,object,int16,int16
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
%%time

# ddf = ddf.repartition(npartitions=100)
# ddf = ddf[ddf['JobText'].notnull()]

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.77 µs


In [9]:
%%time

ddf.head()

CPU times: user 75.1 ms, sys: 8.61 ms, total: 83.7 ms
Wall time: 1.9 s


,Language,CanonCity,CanonState,CleanJobTitle,JobDate,JobID,Latitude,Longitude,CanonPostalCode,CanonCounty,DivisionCode,LMA,MSA,CanonEmployer,CanonJobTitle,ConsolidatedONET,CanonIntermediary,InternshipFlag,Source,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,CanonJobHours,CanonJobType,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedDegreeLevels,ConsolidatedTitle,MaxDegreeLevel,MinDegreeLevel,BGTSubOcc,YearsOfExperience,titles_uni_500,titles_bi_500,titles_uni_1000,titles_bi_1000,dist__bi_1000,dist__uni_500,dist__bi_500,dist__uni_1000,dist__bi_1000_dummy,dist__uni_500_dummy,dist__bi_500_dummy,dist__uni_1000_dummy,clean_text,charac_text_len,len_job_text
0,en,Allentown,PA,Advertising Account Executive Key Accounts,2016-07-08,38087992061,40.6023,-75.4707,18101,Lehigh,NaN,MT421090,10900: Metropolitan Statistical Area,Morning Call,Advertising Account Executive,41301100,NaN,0,Job Board,Marketing and Public Relations: Advertising;Sp...,{'Advertising': 'Marketing and Public Relation...,False,Bachelor's,Bachelor's,NaN,2.0,NaN,41-3011.00,NaN,NaN,NaN,NaN,fulltime,permanent,1-6,mid,16,Advertising Account Executive,16.0,16.0,Advertising Account Executive,At least 2-4 years|2 to 3 years,156,56,995,286,3631.16,1371.38,1783.33,2770.52,0,0,0,0,Advertising Account Executive (Key Accounts) T...,4633,684
1,en,Hanover,NJ,Crop Protection Sales Specialist For Bayer Cro...,2018-06-14,38379581429,40.8218,-74.422,07981,Morris,35084.0,DV343508|MT363562,35620: Metropolitan Statistical Area|408: Comb...,Bayer Corporation,Sales Specialist,41401200,NaN,0,Company,Finance: Budget Management;Specialized Skills|...,{'Budgeting': 'Finance: Budget Management;Spec...,False,NaN,NaN,NaN,NaN,325412.0,41-4011.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sales Specialist,NaN,NaN,Sales Representative,NaN,0,486,581,196,4453.73,1814.80,2200.47,3651.45,0,0,0,0,. Crop Protection Sales Specialist for Ba...,3266,435
2,en,Milwaukee,WI,Program And Policy Chief Performance Improveme...,2017-03-27,38202025999,43.0343,-87.9151,53201,Milwaukee,NaN,MT553334,33340: Metropolitan Statistical Area,Wisdot,NaN,11919900,NaN,0,Job Board,Finance: Budget Management;Specialized Skills|...,{'Budgeting': 'Finance: Budget Management;Spec...,False,NaN,NaN,NaN,NaN,NaN,11-9199.00,90209.0,43.37,62400.0,30.0,fulltime,permanent,NaN,NaN,NaN,Chief Program Performance Manager,NaN,NaN,Program Officer,NaN,313,158,58,607,3851.57,1508.76,1898.18,3053.94,0,0,0,0,In partnership with Hispanic Professionals of ...,5141,740
3,en,West Caldwell,NJ,Machine Operator/Nutraceutical,2020-07-17,38815921335,40.8069,-74.2778,07006,Essex,35084.0,DV343508|MT363562,35620: Metropolitan Statistical Area|408: Comb...,Vitaquest,Machine Operator,51919900,NaN,0,Job Board,Science and Research: Biopharmaceutical Manufa...,{'Batch Records': 'Science and Research: Bioph...,False,Higher Secondary Certificate,Higher Secondary Certificate,NaN,1.0,NaN,51-9012.92,NaN,NaN,NaN,NaN,fulltime,permanent,0-1,low,12,Machine Operator,NaN,12.0,Manufacturing Machine Operator,Minimum of 1 year,212,227,653,642,2668.94,1062.01,1292.53,2162.32,0,0,0,0,Machine Operator / Nutraceutical Vitaquest - W...,3417,516
4,en,Houston,TX,Sales Process Coordinator - Midmarket,2018-04-25,38353595694,30.0683,-95.2163,77325,Harris,NaN,MT482642,26420: Metropolitan Statistical Area,Recruiting Agency,Sales Coordinator,41401200,Insperity,0,Job intermediary,Business: Business Consulting;Specialized Skil...,{'Business Consulting': 'Business: Business Co...,False,Higher Secondary Certificate,Higher Secondary Certificate,NaN,2.0,NaN,41-4011.00,NaN,NaN,NaN,NaN,NaN,NaN,1-6,mid,12,Sales Coordinator,NaN,12.0,Sales Coordinator,One to two-years,315,232,327,756,3160.85,1260.74,1544.18,2553.99,0,0,0,0,Sales Process Coordinator - MidMarket - Job Ti...,3045,452


In [10]:
%%time

ddf.tail()

CPU times: user 43.6 ms, sys: 5.23 ms, total: 48.8 ms
Wall time: 899 ms


,Language,CanonCity,CanonState,CleanJobTitle,JobDate,JobID,Latitude,Longitude,CanonPostalCode,CanonCounty,DivisionCode,LMA,MSA,CanonEmployer,CanonJobTitle,ConsolidatedONET,CanonIntermediary,InternshipFlag,Source,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,CanonJobHours,CanonJobType,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedDegreeLevels,ConsolidatedTitle,MaxDegreeLevel,MinDegreeLevel,BGTSubOcc,YearsOfExperience,titles_uni_500,titles_bi_500,titles_uni_1000,titles_bi_1000,dist__bi_1000,dist__uni_500,dist__bi_500,dist__uni_1000,dist__bi_1000_dummy,dist__uni_500_dummy,dist__bi_500_dummy,dist__uni_1000_dummy,clean_text,charac_text_len,len_job_text
9053,en,Tampa,FL,Rcm - Accounts Receivable Specialist,2018-01-09,38316465836,27.9827,-82.3402,33601,Hillsborough,NaN,MT124530,45300: Metropolitan Statistical Area,Recruiting Agency,Accounts Receivable Specialist,43-303100,Vaco Staffing,0,Job intermediary,Finance: Accounts Payable and Receivable;Speci...,{'Accounts Payable / Accounts Receivable': 'Fi...,False,Higher Secondary Certificate,Associate's|General Equivalency Degree|Higher ...,NaN,1.0,6221.0,43-3031.00,NaN,NaN,NaN,NaN,NaN,NaN,1-6,mid,14|12,Accounts Receivable Specialist,14.0,12.0,Accounts Receivable / Payable Specialist,1 to 3 years|1-3 years,395,127,150,234,3180.78,1273.09,1553.22,2573.20,0,0,0,0,RCM - A/R Specialist $19/HR -Vaco Staffing Ta...,1234,167
9054,en,Durango,CO,Rcis/Registered Cardiovascular Invasive Specia...,2017-03-02,38189722337,37.3268,-107.831,81301,La Plata,NaN,MC082042,20420: Micropolitan Statistical Area,Centura Health,NaN,29203100,NaN,0,Job Board,Health Care: Emergency and Intensive Care;Spec...,{'Advanced Cardiac Life Support (ACLS)': 'Heal...,False,Bachelor's,Bachelor's,510901,1.0,622110.0,29-2031.00,NaN,NaN,NaN,NaN,fulltime,permanent,0-1,low,16,Registered Cardiovascular Specialist,NaN,16.0,Cardiovascular Technician / Technologist,One year|6 months,395,4,110,813,3317.12,1359.07,1624.23,2745.63,0,0,0,0,RCIS/Registered Cardiovascular Invasive Specia...,3921,531
9055,en,Frederick,MD,"Process Systems Software Test Engineer - , Doc...",2011-10-04,6981341,39.4944,-77.4535,21702,Frederick,43524.0,DV241364|MT114790,47900: Metropolitan Statistical Area|548: Comb...,U7 Equipment,Software Test Engineer,15-119902,NaN,0,Job Board,Engineering: Engineering Software;Specialized ...,{'LabVIEW': 'Engineering: Engineering Software...,False,Bachelor's,Bachelor of Science|Bachelor's|Doctor of Medicine,140101|141001|141901,5.0,62.0,15-1133.00,100000.0,48.08,75000.0,36.06,NaN,NaN,6+,high,16|21,Software Test Engineer,21.0,16.0,Systems Engineer,5-10 years,495,163,919,895,4354.36,1604.40,2149.61,3229.78,0,0,0,0,| | | Employers &raquo; Process Systems So...,3839,557
9056,en,New York,NY,Expert Programmer,2019-04-11,38516931242,40.694,-73.9898,11201,Kings,35614.0,DV363564|MT363562,35620: Metropolitan Statistical Area|408: Comb...,Logicsoft,NaN,15113100,NaN,0,Job Board,NaN,{},False,NaN,NaN,NaN,8.0,NaN,15-1131.91,176800.0,85.0,176800.0,85.0,fulltime,temporary,6+,high,NaN,Programmer,NaN,NaN,Computer Programmer,8 years,340,274,414,170,2470.94,1063.22,1188.75,2166.11,1,0,1,0,Logicsoft Logo M204 expert programmer Applied ...,715,110
9057,en,Cheyenne,WY,Qsr Assistant Manager,2017-01-05,38162819143,41.1245,-104.84,82001,Laramie,NaN,MT561694,16940: Metropolitan Statistical Area,Kroger Company,Assistant Manager,41101100,NaN,0,Job Board,Finance: Budget Management;Specialized Skills|...,{'Budgeting': 'Finance: Budget Management;Spec...,False,Higher Secondary Certificate,NaN,NaN,2.0,445110.0,41-1011.00,NaN,NaN,NaN,NaN,fulltime,permanent,1-6,mid,12,Assistant Manager,12.0,12.0,Assistant Retail Store Manager,2-3 years,115,187,81,93,2785.56,1128.92,1352.04,2306.01,0,0,0,0,Part-Time QSR Assistant Manager Job Type : Pe...,4488,664


In [11]:
ddf.npartitions

58

In [12]:
len(ddf.titles_uni_500.unique().compute())

500

In [13]:
%%time

with ProgressBar():
    missing_values = ddf.isna().sum()
    missing_count = ((missing_values / ddf.index.size) * 100)
    missing_count_pct = missing_count.compute()

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
CPU times: user 6.28 s, sys: 952 ms, total: 7.23 s
Wall time: 1min 25s


In [14]:
missing_count_pct

Language                             0.000000
CanonCity                            1.363583
CanonState                           0.014230
CleanJobTitle                        0.000000
JobDate                              0.000000
JobID                                0.000000
Latitude                             1.358933
Longitude                            1.358933
CanonPostalCode                      1.668761
CanonCounty                          1.405851
DivisionCode                        64.768340
LMA                                  1.663126
MSA                                  2.956401
CanonEmployer                        0.000000
CanonJobTitle                       42.674160
ConsolidatedONET                     4.666445
CanonIntermediary                   85.373259
InternshipFlag                       0.000000
Source                               3.839532
CanonSkillClusters                   2.736042
CanonSkills                          0.000000
IsDuplicate                       

In [18]:
import pyarrow

In [14]:
with ProgressBar():
    unique_occus = df.BGTOcc.unique().compute()
    print(unique_occus[:5])

0    11-2022.00
1    49-9071.91
2    19-1020.01
3    15-1134.93
4    53-3032.00
Name: BGTOcc, dtype: object


In [15]:
client.close()

In [15]:
df.tail()

,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
499995,38687559870,Ct Technologist | Radiology/Ct | | With Rotati...,Eau Claire,WI,2019-12-29,CT Technologist | Radiology/CT | 40hrs/wk | Va...,Company,St Nicholas Hospital,44.7985,-91.4824,NaN,Radiologic Technologist,Eau Claire,NaN,20740: Metropolitan Statistical Area,MT552074,0,29203400,Health Care: Emergency and Intensive Care;Spec...,{'Life Support': 'Health Care: Emergency and I...,FALSE,Bachelor's,NaN,NaN,NaN,6221.0,29-2099.92,NaN,NaN,NaN,NaN,1 Year,fulltime,permanent,54701,NaN,NaN,Radiologic Technologist,en,Radiologic Technician / Technologist,16,16.0,16.0
499996,38669407953,Dishwasher,Morrow,GA,2019-11-24,"Dishwasher\n\nChilis\n\n-\n\nMorrow, GA 30260\...",Job Board,Chili's,33.5854,-84.3271,NaN,Dish Washer,Clayton,NaN,12060: Metropolitan Statistical Area,MT131206,0,35902100,Personal Care and Services: Food and Beverage ...,{'Cooking': 'Personal Care and Services: Food ...,FALSE,NaN,NaN,NaN,NaN,722511.0,35-9021.00,NaN,NaN,NaN,NaN,NaN,fulltime,permanent,30260,0,zero,Dish Washer,en,Dishwasher,NaN,NaN,NaN
499997,38635347958,Music Therapist,Des Moines,IA,2019-10-20,Job Information\n\nYouth Emergency Services & ...,Job Board,Youth Emergency Services & Shelter,41.605,-93.6319,NaN,NaN,Polk,NaN,19780: Metropolitan Statistical Area,MT191978,0,29112500,Common Skills|Marketing and Public Relations: ...,"{'Communication Skills': 'Common Skills', 'Com...",FALSE,Bachelor's,Bachelor's,512305,2.0,62.0,29-1125.00,NaN,NaN,NaN,NaN,Minimum of two years,fulltime,permanent,50301,1-6,mid,Music Therapist,en,Art / Music / Recreational Therapist,16,NaN,16.0
499998,38595586361,Cdl A Flatbed Driver - Regional And Over The Road,New Orleans,LA,2019-10-02,CDL A Flatbed Drivers - Regional and OTR\n\nDe...,Job Board,Decker Truck Line,29.9575,-90.0769,NaN,Class A CDL Truck Driver,Orleans,NaN,35380: Metropolitan Statistical Area,MT223538,0,53303200,NaN,{},FALSE,NaN,NaN,NaN,NaN,484.0,53-3032.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70112,NaN,NaN,Class A CDL Truck Driver,en,Over-the-Road (OTR) Truck Driver,NaN,NaN,NaN
499999,38669475936,Azure Engineer,Schaumburg,IL,2019-11-24,"Azure Engineer\n\nYork Solutions, LLC Schaumbu...",Job intermediary,NaN,42.033,-88.0834,York Solutions,Azure Engineer,Cook,16974.0,16980: Metropolitan Statistical Area|176: Comb...,DV171697|MT171698,0,17219900,Information Technology: Distributed Computing;...,{'Apache Kafka': 'Information Technology: Dist...,FALSE,NaN,NaN,NaN,NaN,NaN,NaN,133819.0,64.34,133819.0,64.34,NaN,fulltime,permanent,60159,NaN,NaN,Azure Engineer,en,NaN,NaN,NaN,NaN


In [17]:
with ProgressBar():
    print(df.map_partitions(len).compute())

[########################################] | 100% Completed |  0.1s
0     20000
1     20000
2     20000
3     20000
4     20000
5     20000
6     20000
7     19999
8     20000
9     20000
10    20000
11    20000
12    20000
13    20000
14    20000
15    20000
16    20000
17    20000
18    20000
19    20000
20    20000
21    20000
22    20000
23    20000
24    20000
dtype: int64
